In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
inventory = pd.read_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/inventory_optimized.parquet')
log = pd.read_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/real_log_for_spartan_optimized.parquet')
customer_cluster = pd.read_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/customer_cluster_info.parquet')

In [ ]:
# log_full = log.merge(customer_cluster, on = 'CustomerID', how = 'left').merge(inventory[['GoodsCode', 'Category']], on = 'GoodsCode')
# log_full

In [4]:
# #인벤토리 original_price랑 로그의 price 다른 것 있는지 확인 : 없음
# log_full_temp = log_full.merge(inventory[['GoodsCode', 'OriginalPrice']], on = 'GoodsCode')
# log_full_temp = log_full_temp.assign(price_diff = log_full_temp['OriginalPrice'] - log_full_temp['Price'])
# log_full_temp['price_diff'].unique()

array([0])

## 일렉트로닉 카테고리에서 곧바로 가격별 클러스터링

In [3]:
inventory_extracted = inventory[['Category', 'GoodsCode', 'OriginalPrice']]
inventory_electronics = inventory_extracted[inventory_extracted['Category']== 'Electronics'].reset_index()
inventory_electronics = inventory_electronics[['Category', 'GoodsCode', 'OriginalPrice']]
inventory_electronics


,Category,GoodsCode,OriginalPrice
0,Electronics,1018,124990
1,Electronics,1023,95940
2,Electronics,1024,82980
3,Electronics,1033,114120
4,Electronics,1038,40180
...,...,...,...
104,Electronics,1493,54600
105,Electronics,1495,137560
106,Electronics,1497,40260
107,Electronics,1498,54950


In [4]:
def get_percentage_cluster(a):
    if a >= 100000:
        return "upper"
    if a >= 60000:
        return "middle"
    else:
        return "lower"

electronics_segment = inventory_electronics.copy()
electronics_segment = electronics_segment.assign(Segment = electronics_segment['OriginalPrice'].apply(get_percentage_cluster))
electronics_segment = electronics_segment.rename(columns = {'OriginalPrice' : 'Price'})

In [116]:
electronics_segment.groupby('Segment')['GoodsCode'].count()

Segment
lower     36
middle    38
upper     35
Name: GoodsCode, dtype: int64

In [6]:
electronics_segment

,Category,GoodsCode,Price,Segment
0,Electronics,1018,124990,upper
1,Electronics,1023,95940,middle
2,Electronics,1024,82980,middle
3,Electronics,1033,114120,upper
4,Electronics,1038,40180,lower
...,...,...,...,...
104,Electronics,1493,54600,lower
105,Electronics,1495,137560,upper
106,Electronics,1497,40260,lower
107,Electronics,1498,54950,lower


## 고객 클러스터 별 Electronic에서의 제품군 확인
#### 구매한 제품군들과 구매 안한 제품군들 확인

In [ ]:
# log_electronics = log_full[log_full['Category']=='Electronics']
# log_electronics_rearranged = log_electronics[['cluster_plus','GoodsCode', 'ActionType', 'Timestamp', 'Price', 'AdID', 'CustomerID']]
# log_electronics_segmented = log_electronics_rearranged.merge(electronics_segment[['GoodsCode', 'Segment']], on = 'GoodsCode')
# log_electronics_segmented


In [8]:
# log_electronics_segmented_rearranged = log_electronics_segmented.sort_values(by = ['CustomerID', 'Timestamp'])
# electronics_adclick = log_electronics_segmented_rearranged[log_electronics_segmented_rearranged['ActionType'] == 'ad_click'].sort_values(by='Timestamp')
# electronics_basket = log_electronics_segmented_rearranged[log_electronics_segmented_rearranged['ActionType']=='basket'].sort_values(by='Timestamp')
# electronics_purchase = log_electronics_segmented_rearranged[log_electronics_segmented_rearranged['ActionType']=='purchase'].sort_values(by='Timestamp')



# 통합 과정부터 중요

In [45]:
# electronic_ad_basket = pd.merge_asof(electronics_adclick, electronics_basket[['CustomerID','GoodsCode', 'Segment', 'Timestamp']], by = 'CustomerID', on = 'Timestamp', tolerance=pd.Timedelta(days=7), direction='forward', suffixes = ('', '_bsk'))
# electronic_ad_basket_purchase = pd.merge_asof(electronic_ad_basket,) electronics_purchase[['CustomerID','GoodsCode', 'Segment', 'Timestamp']], by = 'CustomerID', on = 'Timestamp', tolerance=pd.Timedelta(days=7), direction='forward', suffixes = ('', '_pchs')

In [ ]:
# import numpy as np

# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(Final = np.where(
#     ~electronic_ad_basket_purchase['Segment_pchs'].isnull(), 'purchase',
#     np.where(~electronic_ad_basket_purchase['Segment_bsk'].isnull(), 'basket', 'ad_click')
# ))

# electronic_ad_basket_purchase

In [ ]:
# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(
#     SameGoods = np.where(
#         electronic_ad_basket_purchase['Final'] == 'purchase', 
#         np.where(
#             electronic_ad_basket_purchase['GoodsCode_pchs'] == electronic_ad_basket_purchase['GoodsCode'], 
#             'O',
#             'X'
#         ),
#         np.where(
#             electronic_ad_basket_purchase['Final'] == 'basket',
#             np.where(
#                 electronic_ad_basket_purchase['GoodsCode_bsk'] == electronic_ad_basket_purchase['GoodsCode'], 
#                 'O',
#                 'X'
#             ),
#             'O'
#         )
#     )
# )


# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(
#     SameSegment = np.where(
#         electronic_ad_basket_purchase['Final'] == 'purchase', 
#         np.where(
#             electronic_ad_basket_purchase['Segment_pchs'] == electronic_ad_basket_purchase['Segment'], 
#             'O',
#             'X'
#         ),
#         np.where(
#             electronic_ad_basket_purchase['Final'] == 'basket',
#             np.where(
#                 electronic_ad_basket_purchase['Segment_bsk'] == electronic_ad_basket_purchase['Segment'], 
#                 'O',
#                 'X'
#             ),
#             'O'
#         )
#     )
# )


# electronic_ad_basket_purchase

In [54]:
# electronic_ad_basket_purchase.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/electronic_ad_basket_purchase.parquet')


In [ ]:


# electronic_ad_basket_purchase = pd.read_parquet('electronic_ad_basket_purchase.parquet')
# electronic_ad_basket_purchase

In [ ]:
# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(PurchaseCount = np.where(
#     electronic_ad_basket_purchase['Segment_pchs'].notnull(), 1, 0))

# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(BasketCount = np.where(
#     electronic_ad_basket_purchase['Segment_bsk'].notnull(), 1, electronic_ad_basket_purchase['PurchaseCount']))

# electronic_ad_basket_purchase = electronic_ad_basket_purchase.assign(AdclickCount = 1)

# electronic_ad_basket_purchase



In [ ]:
# elec_data_for_funnel = electronic_ad_basket_purchase[['AdID', 'CustomerID', 'cluster_plus','Segment', 'GoodsCode', 'Price', 'Timestamp', 'AdclickCount', 'BasketCount', 'PurchaseCount', 'Final', 'SameGoods', 'SameSegment']]
# elec_data_for_funnel = elec_data_for_funnel.rename(columns = {'Timestamp' : 'AdclickTimestamp', 'cluster_plus' : 'ClusterPlus', 'AdclickCount' : 'ViewCount'})
# elec_data_for_funnel

In [5]:
log_eval_all_df = pd.read_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/log_eval_all_df.parquet')
log_eval_all = log_eval_all_df.merge(customer_cluster, on = 'CustomerID').merge(inventory[['GoodsCode', 'Category']], on = 'GoodsCode').replace(
    {'ad_click': "1_ad_click", 'ad_view': '2_ad_view', 'ad_basket': '3_ad_basket', 'ad_purchase': '4_ad_purchase'}).rename(
        columns = {'cluster_plus' : 'ClusterPlus', 'eval_type': 'Criteria'})

log_eval_elec = log_eval_all.merge(electronics_segment[['GoodsCode', 'Segment']], on = 'GoodsCode')
log_eval_elec = log_eval_elec[['AdID', 'AdID_ad', 'CustomerID', 'ActionType', 'ClusterPlus', 'Segment','GoodsCode', 'Price', 'Timestamp', 'GoodsCode_ad', 'Criteria']]
log_eval_elec['Criteria'] = log_eval_elec['Criteria']
log_eval_elec

,AdID,AdID_ad,CustomerID,ActionType,ClusterPlus,Segment,GoodsCode,Price,Timestamp,GoodsCode_ad,Criteria
0,NaN,NaN,6,view,4,lower,1486,41930,2022-06-28 06:00:23.166991,-1,not_ad_view
1,NaN,NaN,6,view,4,lower,1486,41930,2022-07-04 11:46:45.166991,-1,not_ad_view
2,NaN,NaN,6,purchase,4,lower,1486,41930,2022-07-05 04:40:30.166991,-1,not_ad_purchase
3,NaN,NaN,908,view,4,lower,1486,41930,2022-03-25 02:06:49.486139,-1,not_ad_view
4,NaN,NaN,908,view,4,lower,1486,41930,2022-03-31 20:06:57.486139,-1,not_ad_view
...,...,...,...,...,...,...,...,...,...,...,...
622727,ad_line,ad_line,898306,1_ad_click,1,upper,1340,119710,2022-04-01 17:04:45.847259,1340,1_ad_click
622728,NaN,ad_line,898334,view,1,upper,1340,119710,2022-05-06 04:55:24.020684,1467,ad_view_other
622729,NaN,NaN,898631,view,2,upper,1340,119710,2022-02-07 08:37:16.486139,-1,not_ad_view
622730,NaN,NaN,898631,view,2,upper,1340,119710,2022-02-28 23:22:50.486139,-1,not_ad_view


In [24]:
log_eval_elec.groupby('Segment')['Price'].sum()
log_eval_all_df['Timestamp'].max()

Timestamp('2023-06-11 10:58:15.127184')

In [7]:
#전체 카테고리별

whole_fnl_by_ctgr = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['Category', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_ctgr = whole_fnl_by_ctgr.assign(AdjustedCount = np.where(
    whole_fnl_by_ctgr['Criteria'] == '2_ad_view',
    whole_fnl_by_ctgr['NetCount'] + whole_fnl_by_ctgr['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_ctgr['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_ctgr['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_ctgr['Criteria'] == '3_ad_basket',
        whole_fnl_by_ctgr['NetCount'] + whole_fnl_by_ctgr['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_ctgr['NetCount']
    )
))


whole_fnl_by_ctgr = whole_fnl_by_ctgr.assign(Adjusted_Ratio = (whole_fnl_by_ctgr['AdjustedCount']/whole_fnl_by_ctgr.groupby('Category')['AdjustedCount'].transform(max)).round(3))

whole_fnl_by_ctgr = whole_fnl_by_ctgr.drop(whole_fnl_by_ctgr[whole_fnl_by_ctgr['Criteria']=='1_ad_click'].index)
whole_fnl_by_ctgr


,Category,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,Books,2_ad_view,6651,45356,1.000
2,Books,3_ad_basket,3166,4877,0.108
3,Books,4_ad_purchase,1711,1711,0.038
5,Clothing,2_ad_view,5586,38307,1.000
6,Clothing,3_ad_basket,2584,4096,0.107
7,Clothing,4_ad_purchase,1512,1512,0.039
9,Electronics,2_ad_view,6411,44240,1.000
10,Electronics,3_ad_basket,3116,4815,0.109
11,Electronics,4_ad_purchase,1699,1699,0.038
13,Health & Beauty,2_ad_view,5511,37829,1.000


In [8]:
#카테고리별 광고 매체별
whole_fnl_by_ctgr_ad = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['Category', 'AdID_ad', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_ctgr_ad = whole_fnl_by_ctgr_ad.assign(AdjustedCount = np.where(
    whole_fnl_by_ctgr_ad['Criteria'] == '2_ad_view',
    whole_fnl_by_ctgr_ad['NetCount'] + whole_fnl_by_ctgr_ad['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_ctgr_ad['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_ctgr_ad['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_ctgr_ad['Criteria'] == '3_ad_basket',
        whole_fnl_by_ctgr_ad['NetCount'] + whole_fnl_by_ctgr_ad['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_ctgr_ad['NetCount']
    )
))


whole_fnl_by_ctgr_ad = whole_fnl_by_ctgr_ad.assign(Adjusted_Ratio = (whole_fnl_by_ctgr_ad['AdjustedCount']/whole_fnl_by_ctgr_ad.groupby(['Category', 'AdID_ad'])['AdjustedCount'].transform(max)).round(3))

whole_fnl_by_ctgr_ad = whole_fnl_by_ctgr_ad.drop(whole_fnl_by_ctgr_ad[whole_fnl_by_ctgr_ad['Criteria']=='1_ad_click'].index)
whole_fnl_by_ctgr_ad

,Category,AdID_ad,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,Books,ad_google,2_ad_view,2197,15178,1.000
2,Books,ad_google,3_ad_basket,1103,1694,0.112
3,Books,ad_google,4_ad_purchase,591,591,0.039
5,Books,ad_line,2_ad_view,2149,14431,1.000
6,Books,ad_line,3_ad_basket,978,1515,0.105
7,Books,ad_line,4_ad_purchase,537,537,0.037
9,Books,ad_meta,2_ad_view,2305,15747,1.000
10,Books,ad_meta,3_ad_basket,1085,1668,0.106
11,Books,ad_meta,4_ad_purchase,583,583,0.037
13,Clothing,ad_google,2_ad_view,1812,12704,1.000


In [9]:
#전체 클러스터별

whole_fnl_by_clstr = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_clstr = whole_fnl_by_clstr.assign(AdjustedCount = np.where(
    whole_fnl_by_clstr['Criteria'] == '2_ad_view',
    whole_fnl_by_clstr['NetCount'] + whole_fnl_by_clstr['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_clstr['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_clstr['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_clstr['Criteria'] == '3_ad_basket',
        whole_fnl_by_clstr['NetCount'] + whole_fnl_by_clstr['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_clstr['NetCount']
    )
))


whole_fnl_by_clstr = whole_fnl_by_clstr.assign(Adjusted_Ratio = (whole_fnl_by_clstr['AdjustedCount']/whole_fnl_by_clstr.groupby('ClusterPlus')['AdjustedCount'].transform(max)).round(2))

whole_fnl_by_clstr = whole_fnl_by_clstr.drop(whole_fnl_by_clstr[whole_fnl_by_clstr['Criteria']=='1_ad_click'].index)
whole_fnl_by_clstr


,ClusterPlus,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,2_ad_view,10079,74536,1.00
2,0,3_ad_basket,4959,6977,0.09
3,0,4_ad_purchase,2018,2018,0.03
5,1,2_ad_view,4566,28896,1.00
6,1,3_ad_basket,2076,3506,0.12
7,1,4_ad_purchase,1430,1430,0.05
9,2,2_ad_view,5321,36685,1.00
10,2,3_ad_basket,2604,4073,0.11
11,2,4_ad_purchase,1469,1469,0.04
13,3,2_ad_view,37,369,1.00


In [10]:
#전체퍼널클러스터별카테고리별
whole_fnl_by_clstr_ctgr = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'Category', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_clstr_ctgr = whole_fnl_by_clstr_ctgr.assign(AdjustedCount = np.where(
    whole_fnl_by_clstr_ctgr['Criteria'] == '2_ad_view',
    whole_fnl_by_clstr_ctgr['NetCount'] + whole_fnl_by_clstr_ctgr['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_clstr_ctgr['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_clstr_ctgr['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_clstr_ctgr['Criteria'] == '3_ad_basket',
        whole_fnl_by_clstr_ctgr['NetCount'] + whole_fnl_by_clstr_ctgr['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_clstr_ctgr['NetCount']
    )
))


whole_fnl_by_clstr_ctgr = whole_fnl_by_clstr_ctgr.assign(Adjusted_Ratio = (
    whole_fnl_by_clstr_ctgr['AdjustedCount']/whole_fnl_by_clstr_ctgr.groupby(['ClusterPlus', 'Category'])['AdjustedCount'].transform(max)).round(2))

whole_fnl_by_clstr_ctgr = whole_fnl_by_clstr_ctgr.drop(whole_fnl_by_clstr_ctgr[whole_fnl_by_clstr_ctgr['Criteria']=='1_ad_click'].index)
whole_fnl_by_clstr_ctgr

,ClusterPlus,Category,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,Books,2_ad_view,2324,17233,1.00
2,0,Books,3_ad_basket,1107,1614,0.09
3,0,Books,4_ad_purchase,507,507,0.03
5,0,Clothing,2_ad_view,2031,14717,1.00
6,0,Clothing,3_ad_basket,967,1403,0.10
...,...,...,...,...,...,...
114,vip,Health & Beauty,3_ad_basket,31,57,0.13
115,vip,Health & Beauty,4_ad_purchase,26,26,0.06
117,vip,Home & Garden,2_ad_view,64,445,1.00
118,vip,Home & Garden,3_ad_basket,32,58,0.13


In [11]:
#전체퍼널클러스터별광고매체별
whole_fnl_by_clstr_ad = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'AdID_ad', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_clstr_ad = whole_fnl_by_clstr_ad.assign(AdjustedCount = np.where(
    whole_fnl_by_clstr_ad['Criteria'] == '2_ad_view',
    whole_fnl_by_clstr_ad['NetCount'] + whole_fnl_by_clstr_ad['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_clstr_ad['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_clstr_ad['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_clstr_ad['Criteria'] == '3_ad_basket',
        whole_fnl_by_clstr_ad['NetCount'] + whole_fnl_by_clstr_ad['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_clstr_ad['NetCount']
    )
))


whole_fnl_by_clstr_ad = whole_fnl_by_clstr_ad.assign(Adjusted_Ratio = (
    whole_fnl_by_clstr_ad['AdjustedCount']/whole_fnl_by_clstr_ad.groupby(['ClusterPlus', 'AdID_ad'])['AdjustedCount'].transform(max)).round(2))

whole_fnl_by_clstr_ad = whole_fnl_by_clstr_ad.drop(whole_fnl_by_clstr_ad[whole_fnl_by_clstr_ad['Criteria']=='1_ad_click'].index)
whole_fnl_by_clstr_ad

,ClusterPlus,AdID_ad,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,ad_google,2_ad_view,3393,25133,1.00
2,0,ad_google,3_ad_basket,1667,2361,0.09
3,0,ad_google,4_ad_purchase,694,694,0.03
5,0,ad_line,2_ad_view,3275,24485,1.00
6,0,ad_line,3_ad_basket,1602,2283,0.09
7,0,ad_line,4_ad_purchase,681,681,0.03
9,0,ad_meta,2_ad_view,3411,24918,1.00
10,0,ad_meta,3_ad_basket,1690,2333,0.09
11,0,ad_meta,4_ad_purchase,643,643,0.03
13,1,ad_google,2_ad_view,1496,9553,1.00


In [12]:
#전체퍼널클러스터별광고매체별카테고리별
whole_fnl_by_clstr_ad_ctgr = log_eval_all[log_eval_all['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'AdID_ad', 'Category', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
whole_fnl_by_clstr_ad_ctgr = whole_fnl_by_clstr_ad_ctgr.assign(AdjustedCount = np.where(
    whole_fnl_by_clstr_ad_ctgr['Criteria'] == '2_ad_view',
    whole_fnl_by_clstr_ad_ctgr['NetCount'] + whole_fnl_by_clstr_ad_ctgr['NetCount'].shift(+1, fill_value = 0) + whole_fnl_by_clstr_ad_ctgr['NetCount'].shift(-1, fill_value = 0) + whole_fnl_by_clstr_ad_ctgr['NetCount'].shift(-2, fill_value = 0),
    np.where(
        whole_fnl_by_clstr_ad_ctgr['Criteria'] == '3_ad_basket',
        whole_fnl_by_clstr_ad_ctgr['NetCount'] + whole_fnl_by_clstr_ad_ctgr['NetCount'].shift(-1, fill_value=0),
        whole_fnl_by_clstr_ad_ctgr['NetCount']
    )
))


whole_fnl_by_clstr_ad_ctgr = whole_fnl_by_clstr_ad_ctgr.assign(Adjusted_Ratio = (
    whole_fnl_by_clstr_ad_ctgr['AdjustedCount']/whole_fnl_by_clstr_ad_ctgr.groupby(['ClusterPlus', 'AdID_ad', 'Category'])['AdjustedCount'].transform(max)).round(2))

whole_fnl_by_clstr_ad_ctgr = whole_fnl_by_clstr_ad_ctgr.drop(whole_fnl_by_clstr_ad_ctgr[whole_fnl_by_clstr_ad_ctgr['Criteria']=='1_ad_click'].index)
whole_fnl_by_clstr_ad_ctgr

,ClusterPlus,AdID_ad,Category,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,ad_google,Books,2_ad_view,741,5698,1.00
2,0,ad_google,Books,3_ad_basket,389,567,0.10
3,0,ad_google,Books,4_ad_purchase,178,178,0.03
5,0,ad_google,Clothing,2_ad_view,659,4888,1.00
6,0,ad_google,Clothing,3_ad_basket,337,480,0.10
...,...,...,...,...,...,...,...
354,vip,ad_meta,Health & Beauty,3_ad_basket,12,25,0.14
355,vip,ad_meta,Health & Beauty,4_ad_purchase,13,13,0.07
357,vip,ad_meta,Home & Garden,2_ad_view,18,137,1.00
358,vip,ad_meta,Home & Garden,3_ad_basket,13,14,0.10


## electronics 퍼널

In [13]:
#세그먼트별
elec_fnl_by_seg = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['Segment', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_seg = elec_fnl_by_seg.assign(AdjustedCount = np.where(
    elec_fnl_by_seg['Criteria'] == '2_ad_view',
    elec_fnl_by_seg['NetCount'] + elec_fnl_by_seg['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_seg['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_seg['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_seg['Criteria'] == '3_ad_basket',
        elec_fnl_by_seg['NetCount'] + elec_fnl_by_seg['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_seg['NetCount']
    )
))


elec_fnl_by_seg = elec_fnl_by_seg.assign(Adjusted_Ratio = (elec_fnl_by_seg['AdjustedCount']/elec_fnl_by_seg.groupby('Segment')['AdjustedCount'].transform(max)).round(3))

elec_fnl_by_seg = elec_fnl_by_seg.drop(elec_fnl_by_seg[elec_fnl_by_seg['Criteria']=='1_ad_click'].index)

elec_fnl_by_seg

,Segment,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,lower,2_ad_view,2110,14589,1.000
2,lower,3_ad_basket,1049,1608,0.110
3,lower,4_ad_purchase,559,559,0.038
5,middle,2_ad_view,2290,15529,1.000
6,middle,3_ad_basket,1087,1685,0.109
7,middle,4_ad_purchase,598,598,0.039
9,upper,2_ad_view,2011,14122,1.000
10,upper,3_ad_basket,980,1522,0.108
11,upper,4_ad_purchase,542,542,0.038


In [14]:
#세그먼트별광고매체별

elec_fnl_by_seg_ad = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['Segment', 'AdID_ad', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_seg_ad = elec_fnl_by_seg_ad.assign(AdjustedCount = np.where(
    elec_fnl_by_seg_ad['Criteria'] == '2_ad_view',
    elec_fnl_by_seg_ad['NetCount'] + elec_fnl_by_seg_ad['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_seg_ad['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_seg_ad['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_seg_ad['Criteria'] == '3_ad_basket',
        elec_fnl_by_seg_ad['NetCount'] + elec_fnl_by_seg_ad['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_seg_ad['NetCount']
    )
))


elec_fnl_by_seg_ad = elec_fnl_by_seg_ad.assign(Adjusted_Ratio = (elec_fnl_by_seg_ad['AdjustedCount']/elec_fnl_by_seg_ad.groupby(['Segment', 'AdID_ad'])['AdjustedCount'].transform(max)).round(3))

elec_fnl_by_seg_ad = elec_fnl_by_seg_ad.drop(elec_fnl_by_seg_ad[elec_fnl_by_seg_ad['Criteria']=='1_ad_click'].index)

elec_fnl_by_seg_ad

,Segment,AdID_ad,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,lower,ad_google,2_ad_view,742,4995,1.000
2,lower,ad_google,3_ad_basket,335,533,0.107
3,lower,ad_google,4_ad_purchase,198,198,0.040
5,lower,ad_line,2_ad_view,712,5096,1.000
6,lower,ad_line,3_ad_basket,402,604,0.119
7,lower,ad_line,4_ad_purchase,202,202,0.040
9,lower,ad_meta,2_ad_view,656,4498,1.000
10,lower,ad_meta,3_ad_basket,312,471,0.105
11,lower,ad_meta,4_ad_purchase,159,159,0.035
13,middle,ad_google,2_ad_view,764,5391,1.000


In [15]:


elec_fnl_by_clstr = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_clstr = elec_fnl_by_clstr.assign(AdjustedCount = np.where(
    elec_fnl_by_clstr['Criteria'] == '2_ad_view',
    elec_fnl_by_clstr['NetCount'] + elec_fnl_by_clstr['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_clstr['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_clstr['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_clstr['Criteria'] == '3_ad_basket',
        elec_fnl_by_clstr['NetCount'] + elec_fnl_by_clstr['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_clstr['NetCount']
    )
))


elec_fnl_by_clstr = elec_fnl_by_clstr.assign(Adjusted_Ratio = (elec_fnl_by_clstr['AdjustedCount']/elec_fnl_by_clstr.groupby('ClusterPlus')['AdjustedCount'].transform(max)).round(2))

elec_fnl_by_clstr = elec_fnl_by_clstr.drop(elec_fnl_by_clstr[elec_fnl_by_clstr['Criteria']=='1_ad_click'].index)

elec_fnl_by_clstr



,ClusterPlus,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,2_ad_view,1882,14104,1.00
2,0,3_ad_basket,944,1208,0.09
3,0,4_ad_purchase,264,264,0.02
5,1,2_ad_view,1174,7447,1.00
6,1,3_ad_basket,536,943,0.13
7,1,4_ad_purchase,407,407,0.05
9,2,2_ad_view,1803,12688,1.00
10,2,3_ad_basket,887,1597,0.13
11,2,4_ad_purchase,710,710,0.06
13,3,2_ad_view,11,99,1.00


In [16]:
elec_fnl_by_clstr_ad = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'AdID_ad', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_clstr_ad = elec_fnl_by_clstr_ad.assign(AdjustedCount = np.where(
    elec_fnl_by_clstr_ad['Criteria'] == '2_ad_view',
    elec_fnl_by_clstr_ad['NetCount'] + elec_fnl_by_clstr_ad['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_clstr_ad['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_clstr_ad['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_clstr_ad['Criteria'] == '3_ad_basket',
        elec_fnl_by_clstr_ad['NetCount'] + elec_fnl_by_clstr_ad['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_clstr_ad['NetCount']
    )
))


elec_fnl_by_clstr_ad = elec_fnl_by_clstr_ad.assign(Adjusted_Ratio = (elec_fnl_by_clstr_ad['AdjustedCount']/elec_fnl_by_clstr_ad.groupby(['ClusterPlus', 'AdID_ad'])['AdjustedCount'].transform(max)).round(2))

elec_fnl_by_clstr_ad = elec_fnl_by_clstr_ad.drop(elec_fnl_by_clstr_ad[elec_fnl_by_clstr_ad['Criteria']=='1_ad_click'].index)

elec_fnl_by_clstr_ad


,ClusterPlus,AdID_ad,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,ad_google,2_ad_view,667,4927,1.00
2,0,ad_google,3_ad_basket,307,405,0.08
3,0,ad_google,4_ad_purchase,98,98,0.02
5,0,ad_line,2_ad_view,622,4713,1.00
6,0,ad_line,3_ad_basket,340,424,0.09
7,0,ad_line,4_ad_purchase,84,84,0.02
9,0,ad_meta,2_ad_view,593,4464,1.00
10,0,ad_meta,3_ad_basket,297,379,0.08
11,0,ad_meta,4_ad_purchase,82,82,0.02
13,1,ad_google,2_ad_view,417,2645,1.00


In [17]:
elec_fnl_by_clstr_seg = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'Segment', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_clstr_seg = elec_fnl_by_clstr_seg.assign(AdjustedCount = np.where(
    elec_fnl_by_clstr_seg['Criteria'] == '2_ad_view',
    elec_fnl_by_clstr_seg['NetCount'] + elec_fnl_by_clstr_seg['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_clstr_seg['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_clstr_seg['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_clstr_seg['Criteria'] == '3_ad_basket',
        elec_fnl_by_clstr_seg['NetCount'] + elec_fnl_by_clstr_seg['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_clstr_seg['NetCount']
    )
))


elec_fnl_by_clstr_seg = elec_fnl_by_clstr_seg.assign(Adjusted_Ratio = (elec_fnl_by_clstr_seg['AdjustedCount']/elec_fnl_by_clstr_seg.groupby(['ClusterPlus', 'Segment'])['AdjustedCount'].transform(max)).round(2))

elec_fnl_by_clstr_seg = elec_fnl_by_clstr_seg.drop(elec_fnl_by_clstr_seg[elec_fnl_by_clstr_seg['Criteria']=='1_ad_click'].index)

elec_fnl_by_clstr_seg

,ClusterPlus,Segment,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,lower,2_ad_view,709,5376,1.00
2,0,lower,3_ad_basket,353,503,0.09
3,0,lower,4_ad_purchase,150,150,0.03
5,0,middle,2_ad_view,664,4976,1.00
6,0,middle,3_ad_basket,328,413,0.08
7,0,middle,4_ad_purchase,85,85,0.02
9,0,upper,2_ad_view,509,3752,1.00
10,0,upper,3_ad_basket,263,292,0.08
11,0,upper,4_ad_purchase,29,29,0.01
13,1,lower,2_ad_view,343,2199,1.00


In [18]:
elec_fnl_by_clstr_ad_seg = log_eval_elec[log_eval_elec['Criteria'].isin([
    '1_ad_click', '2_ad_view', '3_ad_basket', '4_ad_purchase'])].groupby(['ClusterPlus', 'AdID_ad', 'Segment', 'Criteria'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'NetCount'})
elec_fnl_by_clstr_ad_seg = elec_fnl_by_clstr_ad_seg.assign(AdjustedCount = np.where(
    elec_fnl_by_clstr_ad_seg['Criteria'] == '2_ad_view',
    elec_fnl_by_clstr_ad_seg['NetCount'] + elec_fnl_by_clstr_ad_seg['NetCount'].shift(+1, fill_value = 0) + elec_fnl_by_clstr_ad_seg['NetCount'].shift(-1, fill_value = 0) + elec_fnl_by_clstr_ad_seg['NetCount'].shift(-2, fill_value = 0),
    np.where(
        elec_fnl_by_clstr_ad_seg['Criteria'] == '3_ad_basket',
        elec_fnl_by_clstr_ad_seg['NetCount'] + elec_fnl_by_clstr_ad_seg['NetCount'].shift(-1, fill_value=0),
        elec_fnl_by_clstr_ad_seg['NetCount']
    )
))


elec_fnl_by_clstr_ad_seg = elec_fnl_by_clstr_ad_seg.assign(Adjusted_Ratio = (elec_fnl_by_clstr_ad_seg['AdjustedCount']/elec_fnl_by_clstr_ad_seg.groupby(['ClusterPlus', 'AdID_ad', 'Segment'])['AdjustedCount'].transform(max)).round(2))

elec_fnl_by_clstr_ad_seg = elec_fnl_by_clstr_ad_seg.drop(elec_fnl_by_clstr_ad_seg[elec_fnl_by_clstr_ad_seg['Criteria']=='1_ad_click'].index)

elec_fnl_by_clstr_ad_seg

,ClusterPlus,AdID_ad,Segment,Criteria,NetCount,AdjustedCount,Adjusted_Ratio
1,0,ad_google,lower,2_ad_view,249,1785,1.00
2,0,ad_google,lower,3_ad_basket,106,157,0.09
3,0,ad_google,lower,4_ad_purchase,51,51,0.03
5,0,ad_google,middle,2_ad_view,218,1735,1.00
6,0,ad_google,middle,3_ad_basket,108,142,0.08
...,...,...,...,...,...,...,...
210,vip,ad_meta,middle,3_ad_basket,7,11,0.12
211,vip,ad_meta,middle,4_ad_purchase,4,4,0.04
213,vip,ad_meta,upper,2_ad_view,25,209,1.00
214,vip,ad_meta,upper,3_ad_basket,16,34,0.16


In [79]:
whole_fnl_by_ctgr.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_ctgr.csv')
whole_fnl_by_ctgr_ad.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_ctgr_ad.csv')
whole_fnl_by_clstr.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_clstr.csv')
whole_fnl_by_clstr_ctgr.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_clstr_ctgr.csv')
whole_fnl_by_clstr_ad.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_clstr_ad.csv')
whole_fnl_by_clstr_ad_ctgr.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/whole_fnl_by_clstr_ad_ctgr.csv')


elec_fnl_by_seg.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_seg.csv')
elec_fnl_by_seg_ad.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_seg_ad.csv')
elec_fnl_by_clstr.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_clstr.csv')
elec_fnl_by_clstr_ad.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_clstr_ad.csv')
elec_fnl_by_clstr_seg.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_clstr_seg.csv')
elec_fnl_by_clstr_ad_seg.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_fnl_by_clstr_ad_seg.csv')

# RFM/P 기반 CLTV 구하기

In [16]:
electronics_segment

,Category,GoodsCode,Price,Segment
0,Electronics,1018,124990,upper
1,Electronics,1023,95940,middle
2,Electronics,1024,82980,middle
3,Electronics,1033,114120,upper
4,Electronics,1038,40180,lower
...,...,...,...,...
104,Electronics,1493,54600,lower
105,Electronics,1495,137560,upper
106,Electronics,1497,40260,lower
107,Electronics,1498,54950,lower


In [29]:
inventory_elec_seg_merged = inventory.merge(electronics_segment[['Category', 'GoodsCode', 'Segment']], on = ['Category', 'GoodsCode'], how = 'left')
inventory_elec_seg_merged

,GoodsCode,Category,OriginalPrice,Inventory,Segment
0,1000,Books,12580,4452,NaN
1,1001,Health & Beauty,21530,2308,NaN
2,1002,Home & Garden,22800,5177,NaN
3,1003,Health & Beauty,22910,9705,NaN
4,1004,Health & Beauty,12780,7852,NaN
...,...,...,...,...,...
495,1495,Electronics,137560,4714,upper
496,1496,Home & Garden,10520,1529,NaN
497,1497,Electronics,40260,3958,lower
498,1498,Electronics,54950,8904,lower


In [30]:
log_eval_all = log_eval_all_df.merge(customer_cluster, on = 'CustomerID', how = 'left').merge(
    inventory_elec_seg_merged[['GoodsCode', 'Category', 'Segment']], on = 'GoodsCode', how = 'left').rename(columns = {'cluster_plus' : 'ClusterPlus'})

log_for_clv = log_eval_all[['ClusterPlus', 'CustomerID', 'Timestamp', 'ActionType', 'Category', 'Segment', 'GoodsCode', 'Price']]
log_for_clv['Timestamp'] = pd.to_datetime(log_for_clv['Timestamp'].dt.date)
log_for_clv = log_for_clv[log_for_clv['ActionType']=='purchase']
log_for_clv

,ClusterPlus,CustomerID,Timestamp,ActionType,Category,Segment,GoodsCode,Price
3,4,6,2022-04-26,purchase,Home & Garden,NaN,1437,35290
10,4,6,2022-06-02,purchase,Health & Beauty,NaN,1047,6670
14,4,6,2022-07-05,purchase,Electronics,lower,1486,41930
17,2,8,2022-03-08,purchase,Clothing,NaN,1372,24930
22,2,8,2022-04-04,purchase,Clothing,NaN,1013,16210
...,...,...,...,...,...,...,...,...
3096910,4,899999,2022-03-18,purchase,Books,NaN,1368,6660
3096911,4,899999,2022-03-21,purchase,Books,NaN,1368,6660
3096914,4,899999,2022-04-04,purchase,Clothing,NaN,1059,21230
3096922,4,899999,2022-04-29,purchase,Clothing,NaN,1059,21230


In [31]:
log_for_clv.groupby(['Category', 'Segment'])['Price'].sum().reset_index()

,Category,Segment,Price
0,Books,lower,0
1,Books,middle,0
2,Books,upper,0
3,Clothing,lower,0
4,Clothing,middle,0
5,Clothing,upper,0
6,Electronics,lower,1505203650
7,Electronics,middle,2829281980
8,Electronics,upper,3889012440
9,Health & Beauty,lower,0


# 클러스터-카테고리별 CLV

In [32]:
#t : 전체 관찰 기간의 마지막 날과 고객의 첫 구매일 차이
#r : 고객의 첫 구매와 마지막 구매일의 차이

t_for_clv = log_for_clv.groupby(['CustomerID', 'Category'])['Timestamp'].min().reset_index().rename(columns = {'Timestamp':'FirstPurchase'})
t_for_clv = t_for_clv.assign(T = (log_for_clv['Timestamp'].max() - t_for_clv['FirstPurchase']).dt.days+1)
r_for_clv = log_for_clv.groupby(['CustomerID', 'Category'])['Timestamp'].max().reset_index().rename(columns = {'Timestamp':'LastPurchase'})
f_for_clv = log_for_clv.groupby(['CustomerID', 'Category'])['Timestamp'].count().reset_index().rename(columns = {'Timestamp':'F'})
m_for_clv = log_for_clv.groupby(['CustomerID', 'Category'])['Price'].sum().reset_index().rename(columns={'Price':'M_whole'})

In [33]:
r_for_clv

,CustomerID,Category,LastPurchase
0,6,Books,NaT
1,6,Clothing,NaT
2,6,Electronics,2022-07-05
3,6,Health & Beauty,2022-06-02
4,6,Home & Garden,2022-04-26
...,...,...,...
882235,899999,Books,2022-03-21
882236,899999,Clothing,2022-04-29
882237,899999,Electronics,NaT
882238,899999,Health & Beauty,NaT


In [34]:
rfmt_for_temp = f_for_clv.merge(r_for_clv, on = ['CustomerID', 'Category'], how = 'left').merge(
    t_for_clv, on = ['CustomerID', 'Category'], how = 'left').merge(m_for_clv, on = ['CustomerID', 'Category'], how = 'left')
rfmt_for_temp = rfmt_for_temp.assign(R = (rfmt_for_temp['LastPurchase'] - rfmt_for_temp['FirstPurchase']).dt.days+1)
rfmt_for_temp = rfmt_for_temp.assign(monetary = (rfmt_for_temp['M_whole']/rfmt_for_temp['F']).round(2))
rfmt_for_temp = rfmt_for_temp.merge(customer_cluster, on = 'CustomerID').rename(columns = {'cluster_plus':'ClusterPlus'})
rfmt_for_clv = rfmt_for_temp[['ClusterPlus', 'CustomerID', 'Category', 'R', 'F', 'monetary', 'T']]
rfmt_for_clv['R'] = (rfmt_for_clv['R']/7).round(2)
rfmt_for_clv['T'] = (rfmt_for_clv['T']/7).round(2)
rfmt_for_clv

,ClusterPlus,CustomerID,Category,R,F,monetary,T
0,4,6,Books,NaN,0,NaN,NaN
1,4,6,Clothing,NaN,0,NaN,NaN
2,4,6,Electronics,0.14,1,41930.0,45.43
3,4,6,Health & Beauty,0.14,1,6670.0,50.14
4,4,6,Home & Garden,0.14,1,35290.0,55.43
...,...,...,...,...,...,...,...
882235,4,899999,Books,0.57,2,6660.0,61.00
882236,4,899999,Clothing,3.71,2,21230.0,58.57
882237,4,899999,Electronics,NaN,0,NaN,NaN
882238,4,899999,Health & Beauty,NaN,0,NaN,NaN


In [35]:
from lifetimes import GammaGammaFitter, BetaGeoFitter

rfmt_for_clv_extracted = rfmt_for_clv[rfmt_for_clv['F'] > 0]

bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(rfmt_for_clv_extracted['F'],
        rfmt_for_clv_extracted['R'],
        rfmt_for_clv_extracted['T'])



ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(rfmt_for_clv_extracted['F'], rfmt_for_clv_extracted['monetary'])


# 고객별 예상 CLV 계산
rfmt_for_clv['CLV12Months'] = ggf.customer_lifetime_value(bgf,
    frequency=rfmt_for_clv_extracted['F'],
    recency=rfmt_for_clv_extracted['R'],
    T=rfmt_for_clv_extracted['T'],
    monetary_value=rfmt_for_clv_extracted['monetary'],
    freq = 'W',
    time=12,  # 12개월 동안의 예측 CLV
    discount_rate=0.01  # 할인율
)

In [36]:
rfmt_for_clv = rfmt_for_clv.rename(columns = {'CLV6Months' : 'CLV12Months'})
rfmt_for_clv

,ClusterPlus,CustomerID,Category,R,F,monetary,T,CLV12Months
0,4,6,Books,NaN,0,NaN,NaN,NaN
1,4,6,Clothing,NaN,0,NaN,NaN,NaN
2,4,6,Electronics,0.14,1,41930.0,45.43,0.516004
3,4,6,Health & Beauty,0.14,1,6670.0,50.14,0.063591
4,4,6,Home & Garden,0.14,1,35290.0,55.43,0.258186
...,...,...,...,...,...,...,...,...
882235,4,899999,Books,0.57,2,6660.0,61.00,0.006312
882236,4,899999,Clothing,3.71,2,21230.0,58.57,3.112701
882237,4,899999,Electronics,NaN,0,NaN,NaN,NaN
882238,4,899999,Health & Beauty,NaN,0,NaN,NaN,NaN


In [78]:
clstr_category_clv = rfmt_for_clv.groupby(['ClusterPlus','Category'])['CLV12Months'].sum().reset_index()
category_clv = clstr_category_clv.groupby('Category')['CLV12Months'].sum().apply(lambda x: "{:,.2f}".format(x)).reset_index()


clstr_category_clv.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/clstr_category_clv.csv')
category_clv.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/category_clv.csv')

In [ ]:
import plotly.graph_objects as go

# 카테고리별 CLV12Months의 합계 계산
category_clv_sum = rfmt_for_clv.groupby('Category')['CLV12Months'].sum()

# 파이 차트 그리기
fig = go.Figure(data=[go.Pie(labels=category_clv_sum.index, values=category_clv_sum.values)])

# 그래프 레이아웃 설정
fig.update_layout(title_text="카테고리별 CLV12Months 합계")

# 그래프 출력
fig.show()

In [ ]:
print(CLV.head())

# 클러스터 - 세그먼트별 CLV

In [40]:
elec_log_for_clv = log_for_clv[log_for_clv['Segment']!='None']

In [41]:
elec_t_for_clv = elec_log_for_clv.groupby(['CustomerID', 'Segment'])['Timestamp'].min().reset_index().rename(columns = {'Timestamp':'FirstPurchase'})
elec_t_for_clv = elec_t_for_clv.assign(T = (elec_log_for_clv['Timestamp'].max() - elec_t_for_clv['FirstPurchase']).dt.days+1)
elec_r_for_clv = elec_log_for_clv.groupby(['CustomerID', 'Segment'])['Timestamp'].max().reset_index().rename(columns = {'Timestamp':'LastPurchase'})
elec_f_for_clv = elec_log_for_clv.groupby(['CustomerID', 'Segment'])['Timestamp'].count().reset_index().rename(columns = {'Timestamp':'F'})
elec_m_for_clv = elec_log_for_clv.groupby(['CustomerID', 'Segment'])['Price'].sum().reset_index().rename(columns={'Price':'M_whole'})

In [299]:
elec_t_for_clv

,CustomerID,Segment,FirstPurchase,T
0,6,lower,2022-07-05,292
1,8,upper,2022-05-06,352
2,17,upper,2022-05-16,342
3,32,lower,2022-03-31,388
4,32,middle,2022-04-04,384
...,...,...,...,...
88519,899952,lower,2022-06-03,324
88520,899956,middle,2022-03-01,418
88521,899956,upper,2022-05-20,338
88522,899992,lower,2022-02-25,422


In [42]:
elec_rfmt_for_temp = elec_f_for_clv.merge(elec_r_for_clv, on = ['CustomerID', 'Segment'], how = 'left').merge(
    elec_t_for_clv, on = ['CustomerID', 'Segment'], how = 'left').merge(elec_m_for_clv, on = ['CustomerID', 'Segment'], how = 'left')
elec_rfmt_for_temp = elec_rfmt_for_temp.assign(R = (elec_rfmt_for_temp['LastPurchase'] - elec_rfmt_for_temp['FirstPurchase']).dt.days+1)
elec_rfmt_for_temp = elec_rfmt_for_temp.assign(monetary = elec_rfmt_for_temp['M_whole']/elec_rfmt_for_temp['F'])
elec_rfmt_for_temp = elec_rfmt_for_temp.merge(customer_cluster, on = 'CustomerID').rename(columns = {'cluster_plus':'ClusterPlus'})
elec_rfmt_for_clv = elec_rfmt_for_temp[['ClusterPlus', 'CustomerID', 'Segment', 'R', 'F', 'monetary', 'T']]
elec_rfmt_for_clv['R'] = (elec_rfmt_for_clv['R']/7).round(2)
elec_rfmt_for_clv['T'] = (elec_rfmt_for_clv['T']/7).round(2)

elec_rfmt_for_clv

,ClusterPlus,CustomerID,Segment,R,F,monetary,T
0,4,6,lower,0.14,1,41930.0,45.43
1,2,8,upper,0.14,1,104910.0,54.00
2,1,17,upper,0.14,1,144050.0,52.57
3,2,32,lower,0.14,1,33180.0,59.14
4,2,32,middle,0.14,1,82700.0,58.57
...,...,...,...,...,...,...,...
88519,0,899952,lower,0.14,1,54950.0,50.00
88520,1,899956,middle,0.14,1,78100.0,63.43
88521,1,899956,upper,3.29,2,101300.0,52.00
88522,4,899992,lower,0.14,1,51170.0,64.00


In [43]:
from lifetimes import GammaGammaFitter, BetaGeoFitter

elec_rfmt_for_clv_extracted = elec_rfmt_for_clv[elec_rfmt_for_clv['F'] > 0]

bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(elec_rfmt_for_clv_extracted['F'],
        elec_rfmt_for_clv_extracted['R'],
        elec_rfmt_for_clv_extracted['T'])



ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(elec_rfmt_for_clv_extracted['F'], elec_rfmt_for_clv_extracted['monetary'])


# 고객별 예상 CLV 계산
elec_rfmt_for_clv['CLV12Months'] = ggf.customer_lifetime_value(bgf,
    frequency=elec_rfmt_for_clv_extracted['F'],
    recency=elec_rfmt_for_clv_extracted['R'],
    T=elec_rfmt_for_clv_extracted['T'],
    monetary_value=elec_rfmt_for_clv_extracted['monetary'],
    freq = 'W',
    time=12,  # 12개월 동안의 예측 CLV
    discount_rate=0.01  # 할인율
)

In [76]:
elec_seg_clv = elec_rfmt_for_clv.groupby(['ClusterPlus','Segment'])['CLV12Months'].sum().reset_index()
elec_seg_clv.to_csv('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/시각화 자료들/elec_seg_clv.csv')

# 퍼널 이전 작업 방식

In [94]:
log_full.query('CustomerID == 6')

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Timestamp,cluster_plus,Category
185426,6,view,1415,12980,NaN,NaN,2022-04-22 15:49:44.166991,4,Health & Beauty
440650,6,view,1486,41930,NaN,NaN,2022-07-04 11:46:45.166991,4,Electronics
440651,6,view,1486,41930,NaN,NaN,2022-06-28 06:00:23.166991,4,Electronics
440652,6,purchase,1486,41930,NaN,NaN,2022-07-05 04:40:30.166991,4,Electronics
778787,6,view,1047,6670,NaN,NaN,2022-05-04 12:39:47.166991,4,Health & Beauty
778788,6,purchase,1047,6670,NaN,NaN,2022-06-02 08:53:44.166991,4,Health & Beauty
778789,6,view,1047,6670,NaN,NaN,2022-05-03 20:32:37.166991,4,Health & Beauty
2309533,6,view,1217,16220,NaN,NaN,2022-05-03 14:26:12.811783,4,Books
2309534,6,basket,1217,16220,NaN,NaN,2022-05-18 03:08:13.811783,4,Books
2389703,6,ad_click,1437,35290,ad_google,182.31,2022-05-07 21:00:59.166991,4,Home & Garden


In [74]:
elec_data_for_funnel

,AdID,CustomerID,ClusterPlus,Segment,GoodsCode,Price,AdclickTimestamp,ViewCount,BasketCount,PurchaseCount,Final,SameGoods,SameSegment
177,ad_google,155913,0,lower,1197,34900,2022-02-08 17:53:36.855443,1,1,1,purchase,O,O
279,ad_google,749706,0,lower,1155,33450,2022-02-10 11:41:34.964177,1,1,1,purchase,O,O
789,ad_line,536777,0,upper,1347,104910,2022-02-17 14:34:38.707955,1,1,1,purchase,O,O
1340,ad_google,690530,0,lower,1070,35180,2022-02-22 22:14:09.124007,1,1,1,purchase,O,O
1367,ad_meta,529849,0,middle,1390,90710,2022-02-23 03:33:10.486139,1,1,1,purchase,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36985,ad_line,893007,0,upper,1171,104650,2022-06-28 20:11:19.693997,1,1,1,purchase,O,O
37010,ad_google,20157,0,upper,1163,100790,2022-06-29 12:10:45.855443,1,1,1,purchase,O,O
37140,ad_meta,746238,0,lower,1197,34900,2022-07-02 17:43:00.125798,1,1,1,purchase,O,O
37290,ad_meta,189267,0,middle,1402,80900,2022-07-17 11:09:49.511967,1,1,1,purchase,O,O


In [72]:
print(31100 + 3683*2 + 2661*3)

46449


In [144]:
elec_data_for_funnel.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/elec_data_for_funnel.parquet')

In [73]:
elec_data_for_funnel[elec_data_for_funnel['CustomerID'].notnull()]['ViewCount'].count()


37444

In [56]:
elec_data_for_funnel.query('PurchaseCount == 1').count()

AdID                2661
CustomerID          2661
ClusterPlus         2661
Segment             2661
GoodsCode           2661
Price               2661
AdclickTimestamp    2661
ViewCount           2661
BasketCount         2661
PurchaseCount       2661
Sameness            2661
dtype: int64

In [78]:
print(31100 + 3683 + 2661)

37444


In [14]:
#같은 상품 구매했는지 상관 노
elec_fnl_whole = elec_data_for_funnel.groupby('Final')['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'Net_Count'})

elec_fnl_whole = elec_fnl_whole.assign(
    Adjusted_Count = np.where(
        elec_fnl_whole['Final'] == 'ad_click', 
        elec_fnl_whole['Net_Count'] + elec_fnl_whole['Net_Count'].shift(-1, fill_value=0) + elec_fnl_whole['Net_Count'].shift(-2, fill_value=0),
        np.where(
            elec_fnl_whole['Final'] == 'basket', 
            elec_fnl_whole['Net_Count'] + elec_fnl_whole['Net_Count'].shift(-1, fill_value=0),
            elec_fnl_whole['Net_Count']
        )
    )
)

elec_fnl_whole = elec_fnl_whole.assign(Adjusted_Ratio = (elec_fnl_whole['Adjusted_Count']/elec_fnl_whole['Adjusted_Count'].max()).round(2))


elec_fnl_whole

,Final,Net_Count,Adjusted_Count,Adjusted_Ratio
0,ad_click,31100,37444,1.00
1,basket,3683,6344,0.17
2,purchase,2661,2661,0.07


In [187]:
elec_fnl_whole.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/전체퍼널세그먼트동일노상관.parquet')

# 같은 세그먼트만 집계한 테이블들에서 ad_click 카운트 다 바꿔줘야함...ㅎ

In [15]:
elec_fnl_whole_same_seg = elec_data_for_funnel[elec_data_for_funnel['SameSegment'] == "O"].groupby(
    ['Final', 'SameSegment'])['CustomerID'].count().reset_index().rename(columns = {'CustomerID' : 'Net_Count'})

elec_fnl_whole_same_seg = elec_fnl_whole_same_seg.assign(
    Adjusted_Count = np.where(
        elec_fnl_whole_same_seg['Final'] == 'ad_click', 
        elec_fnl_whole_same_seg['Net_Count'] + elec_fnl_whole_same_seg['Net_Count'].shift(-1, fill_value=0) + elec_fnl_whole_same_seg['Net_Count'].shift(-2, fill_value=0),
        np.where(
            elec_fnl_whole_same_seg['Final'] == 'basket', 
            elec_fnl_whole_same_seg['Net_Count'] + elec_fnl_whole_same_seg['Net_Count'].shift(-1, fill_value=0),
            elec_fnl_whole_same_seg['Net_Count']
        )
    )
)

elec_fnl_whole_same_seg = elec_fnl_whole_same_seg.assign(Adjusted_Ratio = (elec_fnl_whole_same_seg['Adjusted_Count']/elec_fnl_whole_same_seg['Adjusted_Count'].max()).round(2))


elec_fnl_whole_same_seg


,Final,SameSegment,Net_Count,Adjusted_Count,Adjusted_Ratio
0,ad_click,O,31100,36370,1.00
1,basket,O,3237,5270,0.14
2,purchase,O,2033,2033,0.06


In [188]:
elec_fnl_whole_same_seg.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/전체퍼널세그먼트동일.parquet')

In [16]:
#클러스터별 전체 퍼널
elec_funnel_by_cluster = elec_data_for_funnel.groupby(['ClusterPlus','Final'])['Segment'].count().reset_index().rename(columns = {'Segment' : 'Net_Count'})

elec_funnel_by_cluster = elec_funnel_by_cluster.assign(
    Adjusted_Count = np.where(
        elec_funnel_by_cluster['Final'] == 'ad_click', 
        elec_funnel_by_cluster['Net_Count'] + elec_funnel_by_cluster['Net_Count'].shift(-1, fill_value=0) + elec_funnel_by_cluster['Net_Count'].shift(-2, fill_value=0),
        np.where(
            elec_funnel_by_cluster['Final'] == 'basket', 
            elec_funnel_by_cluster['Net_Count'] + elec_funnel_by_cluster['Net_Count'].shift(-1, fill_value=0),
            elec_funnel_by_cluster['Net_Count']
        )
    )
)

elec_funnel_by_cluster = elec_funnel_by_cluster.assign(Adjusted_Ratio = (
    elec_funnel_by_cluster['Adjusted_Count'] / elec_funnel_by_cluster.groupby('ClusterPlus')['Adjusted_Count'].transform('max')).round(2))


In [189]:
elec_funnel_by_cluster.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/클러스터별퍼널세그먼트동일노상관.parquet')

In [19]:
elec_funnel_by_cluster

,ClusterPlus,Final,Net_Count,Adjusted_Count,Adjusted_Ratio
0,0,ad_click,9616,11014,1.00
1,0,basket,1027,1398,0.13
2,0,purchase,371,371,0.03
3,1,ad_click,3993,5330,1.00
4,1,basket,617,1337,0.25
5,1,purchase,720,720,0.14
6,2,ad_click,7273,9288,1.00
7,2,basket,963,2015,0.22
8,2,purchase,1052,1052,0.11
9,3,ad_click,67,77,1.00


In [17]:
elec_fnl_by_clstr_same_seg = elec_data_for_funnel.groupby(['ClusterPlus', 'SameSegment', 'Final'])['Segment'].count().reset_index().rename(columns = {'Segment' : 'Net_Count'})

elec_fnl_by_clstr_same_seg = elec_fnl_by_clstr_same_seg.assign(
    Adjusted_Count = np.where(
        elec_fnl_by_clstr_same_seg['Final'] == 'ad_click', 
        elec_fnl_by_clstr_same_seg['Net_Count'] + elec_fnl_by_clstr_same_seg['Net_Count'].shift(-1, fill_value=0) + elec_fnl_by_clstr_same_seg['Net_Count'].shift(-2, fill_value=0),
        np.where(
            elec_fnl_by_clstr_same_seg['Final'] == 'basket', 
            elec_fnl_by_clstr_same_seg['Net_Count'] + elec_fnl_by_clstr_same_seg['Net_Count'].shift(-1, fill_value=0),
            elec_fnl_by_clstr_same_seg['Net_Count']
        )
    )
)

elec_fnl_by_clstr_same_seg = elec_fnl_by_clstr_same_seg.assign(Adjusted_Ratio = (
    elec_fnl_by_clstr_same_seg['Adjusted_Count'] / elec_fnl_by_clstr_same_seg.groupby(['ClusterPlus', 'SameSegment'])['Adjusted_Count'].transform('max')).round(2))

elec_fnl_by_clstr_same_seg = elec_fnl_by_clstr_same_seg[elec_fnl_by_clstr_same_seg['SameSegment'] == "O"]
elec_fnl_by_clstr_same_seg = elec_fnl_by_clstr_same_seg.reset_index()


In [190]:
elec_fnl_by_clstr_same_seg.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/클러스터별퍼널세그먼트동일.parquet')

In [18]:
elec_fnl_by_clstr_same_seg

,index,ClusterPlus,SameSegment,Final,Net_Count,Adjusted_Count,Adjusted_Ratio
0,0,0,O,ad_click,9616,10854,1.00
1,1,0,O,basket,937,1238,0.11
2,2,0,O,purchase,301,301,0.03
3,5,1,O,ad_click,3993,5006,1.00
4,6,1,O,basket,512,1013,0.20
5,7,1,O,purchase,501,501,0.10
6,10,2,O,ad_click,7273,8935,1.00
7,11,2,O,basket,831,1662,0.19
8,12,2,O,purchase,831,831,0.09
9,15,3,O,ad_click,67,77,1.00


In [ ]:
#전체 퍼널 : 같은 품목인지 count X
elec_seg_fnl_by_cluster = elec_data_for_funnel.groupby(['ClusterPlus', 'Segment', 'Final'])['GoodsCode'].count().reset_index().rename(columns = {'GoodsCode' : 'Net_Count'})

elec_seg_fnl_by_cluster = elec_seg_fnl_by_cluster.assign(
    Adjusted_Count = np.where(
        elec_seg_fnl_by_cluster['Final'] == 'ad_click',
        elec_seg_fnl_by_cluster['Net_Count'] + elec_seg_fnl_by_cluster['Net_Count'].shift(-1, fill_value=0) + elec_seg_fnl_by_cluster['Net_Count'].shift(-2, fill_value=0), 
        np.where(
            elec_seg_fnl_by_cluster['Final'] == 'basket',
            elec_seg_fnl_by_cluster['Net_Count'] + elec_seg_fnl_by_cluster['Net_Count'].shift(-1, fill_value=0),
            elec_seg_fnl_by_cluster['Net_Count']
        )
    )
)


elec_seg_fnl_by_cluster = elec_seg_fnl_by_cluster.assign(Adjusted_Ratio = (
    elec_seg_fnl_by_cluster['Adjusted_Count'] / elec_seg_fnl_by_cluster.groupby(['ClusterPlus', 'Segment'])['Adjusted_Count'].transform('max')).round(2)
)


elec_seg_fnl_by_cluster


In [194]:
elec_seg_fnl_by_cluster

,ClusterPlus,Segment,Final,Net_Count,Adjusted_Count,Adjusted_Ratio
0,0,lower,ad_click,3610,4164,1.00
1,0,lower,basket,371,554,0.13
2,0,lower,purchase,183,183,0.04
3,0,middle,ad_click,3412,3899,1.00
4,0,middle,basket,361,487,0.12
5,0,middle,purchase,126,126,0.03
6,0,upper,ad_click,2594,2951,1.00
7,0,upper,basket,295,357,0.12
8,0,upper,purchase,62,62,0.02
9,1,lower,ad_click,1168,1582,1.00


In [191]:
elec_seg_fnl_by_cluster.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/클러스터별세그먼트퍼널세그먼트동일노상관.parquet')

In [186]:
elec_seg_fnl_by_clst_same_seg = elec_data_for_funnel.groupby(['ClusterPlus','SameSegment', 'Segment', 'Final'])['GoodsCode'].count().reset_index().rename(columns = {'GoodsCode' : 'Net_Count'})

elec_seg_fnl_by_clst_same_seg = elec_seg_fnl_by_clst_same_seg.assign(
    Adjusted_Count = np.where(
        elec_seg_fnl_by_clst_same_seg['Final'] == 'ad_click',
        elec_seg_fnl_by_clst_same_seg['Net_Count'] + elec_seg_fnl_by_clst_same_seg['Net_Count'].shift(-1, fill_value=0) + elec_seg_fnl_by_clst_same_seg['Net_Count'].shift(-2, fill_value=0), 
        np.where(
            elec_seg_fnl_by_clst_same_seg['Final'] == 'basket',
            elec_seg_fnl_by_clst_same_seg['Net_Count'] + elec_seg_fnl_by_clst_same_seg['Net_Count'].shift(-1, fill_value=0),
            elec_seg_fnl_by_clst_same_seg['Net_Count']
        )
    )
)

elec_seg_fnl_by_clst_same_seg = elec_seg_fnl_by_clst_same_seg.assign(Adjusted_Ratio = (
    elec_seg_fnl_by_clst_same_seg['Adjusted_Count'] / elec_seg_fnl_by_clst_same_seg.groupby(['ClusterPlus', 'SameSegment', 'Segment'])['Adjusted_Count'].transform('max')).round(2)
)

elec_seg_fnl_by_clst_same_seg = elec_seg_fnl_by_clst_same_seg[elec_seg_fnl_by_clst_same_seg['SameSegment'] == 'O']
elec_seg_fnl_by_clst_same_seg = elec_seg_fnl_by_clst_same_seg.reset_index()


In [192]:
elec_seg_fnl_by_clst_same_seg.to_parquet('C:/Users/neddy/Desktop/스파르타 코딩클럽/최종 프로젝트/데이터셋 및 코딩 파일/클러스터링기반제품퍼널/퍼널최종파일들/클러스터별세그먼트퍼널세그먼트동일.parquet')


In [193]:
elec_seg_fnl_by_clst_same_seg

,index,ClusterPlus,SameSegment,Segment,Final,Net_Count,Adjusted_Count,Adjusted_Ratio
0,0,0,O,lower,ad_click,3610,4121,1.00
1,1,0,O,lower,basket,342,511,0.12
2,2,0,O,lower,purchase,169,169,0.04
3,3,0,O,middle,ad_click,3412,3843,1.00
4,4,0,O,middle,basket,330,431,0.11
5,5,0,O,middle,purchase,101,101,0.03
6,6,0,O,upper,ad_click,2594,2890,1.00
7,7,0,O,upper,basket,265,296,0.10
8,8,0,O,upper,purchase,31,31,0.01
9,15,1,O,lower,ad_click,1168,1464,1.00
